<a href="https://colab.research.google.com/github/mariabandeira/AdministracaoPublica/blob/main/CadUnico/KNN/TrainningPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

In [3]:
treated_data_RN = pd.read_csv('./BasesRN/cadunico_rn_cleaned.csv', sep=';', encoding='utf-8')

In [4]:
treated_data_RN.dropna(inplace=True)
treated_data_RN.drop_duplicates(inplace=True)

### Treinando o modelo RNN

In [8]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = treated_data_RN['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution, 2))

marc_pbf
1    56.74
0    43.26
Name: proportion, dtype: float64


In [9]:
# separar X (features) e y (target)
X = treated_data_RN.drop('marc_pbf', axis=1)
y = treated_data_RN['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criando uma instância do RandomUnderSampling
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# balanceando os dados
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(y_resampled.value_counts())

marc_pbf
0    89679
1    89679
Name: count, dtype: int64


In [10]:
# aplicar padronização nas features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print("Primeiros dados escalados (padronizados):")
print(X_train_scaled[:5])

Primeiros dados escalados (padronizados):
[[-0.84487543 -1.96467582 -0.12485031 -0.57790393 -0.602914   -1.16411791
  -0.72537277 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997 -0.85437521  0.05656285 -1.37479098]
 [-0.55264356  0.71176597  0.13504909 -0.57790393 -0.602914    0.30866763
  -0.72537277 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997  1.22999908  0.05656285  0.90668609]
 [-0.75095372 -0.62645493  4.64343735  1.7303914   0.1538476  -1.16411791
   1.28227208 -0.28287978 -0.39178363  1.7490837  -0.16908028  2.00891719
  -0.23932997  1.22999908  0.05656285 -1.37479098]
 [ 0.81400362  0.71176597  1.40335816  1.7303914   3.93765562  1.78145316
   1.28227208 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997  0.18781194  0.05656285  0.14619374]
 [ 1.32648826 -0.62645493 -0.81791538 -0.57790393  0.91060921  0.30866763
   1.28227208 -0.28287978 -0.39178363 -0.45627415 -0.16908028 -0.46355174
  -0.23932997 

In [13]:
# Ajustar formato para a RNN (samples, time steps, features)
X_train_rnn = np.expand_dims(X_train_scaled, axis=1)
X_test_rnn = np.expand_dims(X_test_scaled, axis=1)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [15]:
# Construir a RNN
model = Sequential([
    SimpleRNN(32, activation='relu', input_shape=(1, X_train.shape[1])),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(X_train_rnn, y_resampled, epochs=20, batch_size=32, validation_data=(X_test_rnn, y_test))

Epoch 1/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.8458 - loss: 0.3612 - val_accuracy: 0.8880 - val_loss: 0.3010
Epoch 2/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms/step - accuracy: 0.8846 - loss: 0.3023 - val_accuracy: 0.8871 - val_loss: 0.3023
Epoch 3/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8840 - loss: 0.3049 - val_accuracy: 0.8918 - val_loss: 0.2946
Epoch 4/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8853 - loss: 0.3001 - val_accuracy: 0.8907 - val_loss: 0.3013
Epoch 5/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8868 - loss: 0.2993 - val_accuracy: 0.8911 - val_loss: 0.2962
Epoch 6/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.8882 - loss: 0.2948 - val_accuracy: 0.8913 - val_loss: 0.2959
Epoch 7/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.8873 - loss: 0.2958 - val_accuracy: 0.8917 - val_loss: 0.2959
Epoch 8/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8863 - loss: 0

In [18]:
y_pred_prob = model.predict(X_test_rnn)
y_pred = (y_pred_prob > 0.5).astype(int)

2776/2776 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.83      0.87     38411
           1       0.88      0.94      0.91     50408

    accuracy                           0.89     88819
   macro avg       0.90      0.89      0.89     88819
weighted avg       0.89      0.89      0.89     88819



### Selecionando features com SelectKBests

In [20]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

Características selecionadas: Index(['vlr_renda_media_fam', 'qtd_comodos_domic_fam', 'cod_material_piso_fam',
       'cod_calcamento_domic_fam', 'qtde_pessoas'],
      dtype='object')


In [22]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

In [23]:
X_train_rnn = np.expand_dims(X_train_scaled, axis=1)
X_test_rnn = np.expand_dims(X_test_scaled, axis=1)

In [24]:
model = Sequential([
    SimpleRNN(32, activation='relu', input_shape=(1, X_train_scaled.shape[1])),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(X_train_rnn, y_resampled, epochs=20, batch_size=32, validation_data=(X_test_rnn, y_test))

Epoch 1/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - accuracy: 0.8624 - loss: 0.3410 - val_accuracy: 0.8911 - val_loss: 0.2987
Epoch 2/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - accuracy: 0.8862 - loss: 0.3023 - val_accuracy: 0.8918 - val_loss: 0.2988
Epoch 3/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - accuracy: 0.8869 - loss: 0.3003 - val_accuracy: 0.8902 - val_loss: 0.3026
Epoch 4/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - accuracy: 0.8893 - loss: 0.2964 - val_accuracy: 0.8927 - val_loss: 0.2969
Epoch 5/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.8878 - loss: 0.2980 - val_accuracy: 0.8923 - val_loss: 0.2993
Epoch 6/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 44s 4ms/step - accuracy: 0.8890 - loss: 0.2980 - val_accuracy: 0.8942 - val_loss: 0.2932
Epoch 7/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.8874 - loss: 0.3002 - val_accuracy: 0.8877 - val_loss: 0.3017
Epoch 8/20
5605/5605 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - accuracy: 0.8874 - loss: 0

In [27]:
y_pred_prob = model.predict(X_test_rnn)
y_pred = (y_pred_prob > 0.5).astype(int)

2776/2776 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.83      0.87     38411
           1       0.88      0.95      0.91     50408

    accuracy                           0.90     88819
   macro avg       0.90      0.89      0.89     88819
weighted avg       0.90      0.90      0.89     88819

